<a href="https://colab.research.google.com/github/Varun1723/-AI-POWERED-PLASTIC-WASTE-SEGREGATION-From-Hotspot-Detection-to-Material-Classification/blob/main/Plastic_hotspot_Identify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Libraries
import ee
import geemap

# Initialize GEE
ee.Authenticate()
ee.Initialize(project='ee-vst1723thakur')

# Define the region (India's bounding box)
india = ee.Geometry.Rectangle([68, 8, 97, 37])

# Load Sentinel-2 Harmonized Dataset (With Atmospheric Correction)
sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(india) \
            .filterDate('2023-01-01', '2023-12-31') \
            .median()

# Clip the dataset to India's boundary
sentinel = sentinel.clip(india)

# --------------------------------------------
# Step 1: Land & Water Segmentation Using NDWI
# --------------------------------------------

# Compute NDWI (Normalized Difference Water Index)
def compute_ndwi(image):
    return image.normalizedDifference(['B3', 'B8']).rename('NDWI')

ndwi = compute_ndwi(sentinel)

# Classify Land & Water using NDWI thresholding
water = ndwi.gt(0)   # Water Bodies (Deep Blue)
land = ndwi.lte(0)   # Land Areas (Light Gray)

# Convert classification into an image (1 = Water, 2 = Land)
land_water_classified = water.multiply(1).add(land.multiply(2))

# Define visualization parameters for NDWI
ndwi_viz = {
    'min': 1,
    'max': 2,
    'palette': ['#0000FF', '#D3D3D3'],  # Blue for Water, Gray for Land
}

# --------------------------------------------
# Step 2: Floating Plastic Debris Detection (FDI)
# --------------------------------------------

# Compute FDI (Floating Debris Index)
def compute_fdi(image):
    b8 = image.select('B8')   # Near Infrared (NIR)
    b11 = image.select('B11') # Shortwave Infrared (SWIR)
    b4 = image.select('B4')   # Red Band

    # Floating Debris Index formula
    fdi = b8.subtract(b11.add((b4.subtract(b11)).divide(b4.subtract(b8))))

    return fdi.rename('FDI')

# Apply FDI calculation
fdi_layer = compute_fdi(sentinel)

# Classify FDI into Low, Medium, and High
fdi_high = fdi_layer.gt(0.3)  # High Plastic Waste
fdi_medium = fdi_layer.gt(0.15).And(fdi_layer.lte(0.3))  # Medium Plastic Waste
fdi_low = fdi_layer.gt(0.05).And(fdi_layer.lte(0.15))  # Low Plastic Waste

# Mask Land Areas (Keep Only Water)
fdi_masked = fdi_layer.updateMask(water)  # Hides FDI on land

# Convert classification into an image (1 = Low, 2 = Medium, 3 = High)
fdi_classified = fdi_high.multiply(3).add(fdi_medium.multiply(2)).add(fdi_low.multiply(1)).updateMask(water)

# Define visualization parameters for FDI
fdi_viz = {
    'min': 1,
    'max': 3,
    'palette': ['#FFC0CB', '#FFA500', '#FFFF00'],  # Pink (Low), Orange (Medium), Yellow (High)
    'opacity': 0.6  # Make FDI semi-transparent
}

# --------------------------------------------
# Step 3: Map Visualization
# --------------------------------------------

# Create and Center Map
Map = geemap.Map()
Map.centerObject(india, 5)

# Add Land & Water Layer
Map.addLayer(land_water_classified, ndwi_viz, "Land & Water Segmentation")

# Overlay FDI Layer (Masked to Show Only on Water)
Map.addLayer(fdi_classified, fdi_viz, "Floating Plastic Debris (FDI)")

# Display Map
Map


Map(center=[22.565416328717983, 82.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…